# TP IAR 2020

In [1]:
import numpy as np
import math
import csv
import pickle
import matplotlib.pyplot as plt
import sys
from random import seed
from random import random
from math import exp

In [2]:
#funciones de activación globales, para ser utilizadas más adelante
def tanh(activation):
    return np.tanh(activation)

def tanh_derivative(output):
    return (1.0 - (output*output))

In [3]:
class Neuron:
    """
    Define la estructura básica de neurona, con su respectivo peso
    """
    def __init__(self, weight=None):
        if not weight:
            self.weight = random()
        else:
            self.weight = weight
            
    def __str__(self):
        return str(self.weight)

In [4]:
class Layer:
    """
    Define la unión de las neuronas
    """
    def __init__(self, n, neurons=None):
        if not neurons:
            self.neurons = []
            for i in range(n):
                self.neurons.append(Neuron())
        else:
            self.neurons = neurons

        self.output = None 
        self.delta = None 
        
    def __str__(self):
        neurons_print = "neurons = ["
        for neuron in self.neurons:
            neurons_print = neurons_print + str(neuron) + ", "
        neurons_print = neurons_print[:-2] + "]\n"
        output_print = "output = {}\n".format(self.output)
        delta_print = "delta = {}\n".format(self.delta)

        return neurons_print + output_print + delta_print
                       
    def activate(self, layer, inputs):
        activation = layer[-1].weight
        for i in range(len(layer) -1):
            activation += layer[i].weight * inputs[i]
        return activation

In [5]:
class Clasificador:
    
    def __init__(self, shape):
        """
        arguments: shape se definirá como un diccionario,
               que tiene como elementos el tamaño de la red
        """
        self.layers = list()
        self._init_layers(shape)
        
    def _init_layers(self, shape):
        """
        inicializa las capas según los valores definidos por el usuario
        """
        self.hidden_layer = []
        for i in range(shape["hidden"]):
            self.hidden_layer.append(Layer(shape["inputs"] + 1))
        
        self.output_layer = []
        for i in range(shape["outputs"]):
            self.output_layer.append(Layer(shape["hidden"] + 1))
    
        self.layers.append(self.hidden_layer)
        self.layers.append(self.output_layer)
            
    def __str__(self):
        """
        muestra estructura red
        """
        out = ""
        for super_layer in self.layers:
            out = out + "---- complete layer ----\n"
            for sub_layer in super_layer:
                out = out + "-- sub layer --\n"
                out = out + str(sub_layer)
        return out    
        
    def forward_propagate(self, net_inputs):
        """
        cálculo de la salida (hacia adelante) de las capas hasta el final
        """
        inputs = net_inputs
        for super_layer in self.layers:
            temporal_output = []
            for sub_layer in super_layer:
                activation = sub_layer.activate(sub_layer.neurons, inputs)
                output = tanh(activation) #sigmoid(activation) #tanh(activation)
                sub_layer.output = output
                temporal_output.append(output)

            inputs = temporal_output
        return inputs
        
    def backward_propagate_error(self, expected):
        """
        cálculo de los errores hacia atrás
        """
        net_range = len(self.layers)
        
        for i in reversed(range(net_range)):
            layer = self.layers[i]
            errors = list()
            
            layer_range = len(layer)
            
            if i != (net_range - 1):         
                for j in range(layer_range):
                    error = 0.0
                    for element in self.layers[i+1]:
                        error += element.neurons[j].weight * element.delta
                    errors.append(error)
            else:
                for j in range(layer_range):
                    element = layer[j]
                    output = element.output
                    errors.append(expected[j] - output)
                
            for j in range(layer_range):
                element = layer[j]
                element.delta = errors[j] * tanh_derivative(element.output)
        
    def update_weights(self, net_inputs, learning_rate):
        """
        actualización de los pesos de cada capa
        """
        net_range = len(self.layers)
        
        for i in range(net_range):            
            inputs = net_inputs[:-1]

            if i != 0:
                inputs = [element.output for element in self.layers[i-1]]
                
            for element in self.layers[i]:
                for j in range(len(inputs)):
                    element.neurons[j].weight += learning_rate * element.delta * inputs[j]
                element.neurons[-1].weight += learning_rate * element.delta
    
        
    def fit(self, dataset, learning_rate, epochs, n_outputs):
        """
        entrenamiento de la red
        """
        for epoch in range(epochs):
            sum_error = 0
            for row in dataset:
                outputs = self.forward_propagate(row)
                expected = [0 for i in range(n_outputs)]
                expected[int(row[-1])] = 1
                sum_error += sum([(expected[i]-outputs[i])**2 for i in range(len(expected))])
                self.backward_propagate_error(expected)
                self.update_weights(row, learning_rate)
    
    def predict(self, row):
        """
        predicciones
        """
        outputs = self.forward_propagate(row)
        return outputs.index(max(outputs))
    
    def compute_confusion_matrix(self, y_expected, y_predicted):
        """
        cálculo de la matriz de confusión
        """
        classes = len(np.unique(y_expected))
        result = np.zeros((classes, classes))
    
        for i in range(len(y_expected)):
            result[y_expected[i]][y_predicted[i]] += 1

        return result

    def save_params(self, nombre="params"):
        pickle.dump(self.w, open(nombre + ".pickle", "wb"))
        
    def load_params(self, nombre="params"):
        self.w = pickle.load(open(nombre + ".pickle"), "rb")
        

### Lectura de datos

In [6]:
data_path = './'

X = np.genfromtxt(data_path + 'X.csv', delimiter=',')
Y_plain = Y = np.genfromtxt(data_path + 'Y.csv', delimiter=',')
Y = np.vstack(Y_plain)

dataset = np.concatenate([X, Y], axis = 1)

print(dataset)

[[2.21192195 1.52152313 3.32068152 6.7957075  1.        ]
 [4.64300389 4.11081718 1.07105344 6.3184066  0.        ]
 [9.53565588 4.66845313 4.64456258 6.89257513 1.        ]
 ...
 [3.22427171 4.73382836 3.02922467 5.55866423 0.        ]
 [6.16262185 2.14275167 8.43693035 3.91897483 0.        ]
 [4.2134662  7.25175006 8.64138786 4.84727099 0.        ]]


### Clasificación

In [14]:
n_inputs = len(dataset[0]) - 1
#print("INPUT DIM ")
#print(n_inputs)

n_outputs = len(set([row[-1] for row in dataset]))
#print("OUTPUT DIM ")
#print(n_outputs)

seed(1)

shape = {
    "inputs": n_inputs,
    "hidden": 6,
    "outputs": n_outputs
}
net = Clasificador(shape) #inicializa la red

net.fit(dataset, 0.040, 500, n_outputs) #con learning rate 0.040 y 100 epochs, clasifica 'correctamente' 1821 de 20000

### Evaluación

In [15]:
#abro el set de datos inicial nuevamente para convertir los valores a int (eran float)

y_data = open('./Y.csv')
y_coordinates = csv.reader(y_data, delimiter=',')

y_list = []

for row in y_coordinates:
    y_list.append(row)

new_y_list = []
for sub in y_list:
    new_sub = []
    for item in sub:
        new_item = float(item)
        new_sub.append(int(new_item))
    new_y_list.append(new_sub)

y_array = np.array(new_y_list)
Y = np.hstack(y_array) 


In [16]:
#realizo las predicciones
temporal_list = []

for row in dataset:
    prediction = net.predict(row)
    temporal_list.append(prediction)
    
y_predictions = np.array(temporal_list)

In [17]:
matrix  = net.compute_confusion_matrix(Y, y_predictions) #calculo la matriz de confusión
print(matrix)

[[9306.  694.]
 [ 945. 9055.]]


In [18]:
true_negative = matrix[0][0]
false_positive = matrix[0][1]
false_negative = matrix[1][0]
true_positive = matrix[1][1]

In [19]:
#precision True Positive/(True Positive + False Positive)
precision = true_positive / (true_positive + false_positive)
accuracy = (true_negative + true_positive)/(true_negative + true_positive + false_positive + false_negative)
recall = true_positive / (true_positive + false_negative)
t_negative_rate = false_positive / (false_positive + true_negative) 
print("Precision: ", precision)
print("Accuracy: ", accuracy)
print("Recall: ", recall)
print("True negative rate: ", t_negative_rate)

Precision:  0.9288132116114474
Accuracy:  0.91805
Recall:  0.9055
True negative rate:  0.0694


### Validación de la cátedra y exportación de resultados y parámetros

In [21]:
#realizo las predicciones para X_test.csv

X = np.genfromtxt(data_path + 'X_test.csv', delimiter=',')

temporal_list = []

for row in X:
    prediction = net.predict(row)
    temporal_list.append(prediction)
    
y_predictions = np.array(temporal_list)

In [23]:
np.savetxt('Y_test_Grupo_04.csv', y_predictions, fmt='%.18e', delimiter=',')
